In [1]:
from sklearn.preprocessing import Normalizer
from keras.models import load_model
from keras.utils import img_to_array
import cv2
import time
import csv
import time as tp
from datetime import datetime
from scipy.spatial.distance import cosine
import pickle
import numpy as np
import mtcnn

In [2]:

def record_attendance(names, frame,c):
    # Step 1: Create a dictionary to store the attendance for each date.
    attendance = {}

    # Step 2: Iterate over the names, and check if each name is present in the frame.
    for name in names:
        if name in frame:
            # Step 3: Get the current date and time and add the student to the attendance dictionary.
            now = datetime.now()
            date = now.strftime('%d %b %Y')
            time = now.strftime('%H:%M:%S')
            if date not in attendance:
                attendance[date] = []
            attendance[date].append((name, time))

    # Print out the frame and names variables for debugging
    print("Frame:", frame)
    print("Names:", names)

    # Step 4: Open the CSV file in append mode to add new rows to the existing data.
    with open('attendance{}.csv'.format(c), 'a', newline='') as file:

        # Step 5: Create a csv.writer object.
        writer = csv.writer(file, delimiter=',')

        # Step 6: If the file is empty, write the headers to the CSV file.
        if file.tell() == 0:
            writer.writerow(['Date', 'Name', 'Time', 'Attendance'])

        # Step 7: Write the attendance for each date to the CSV file.
        for date in attendance:
            for student in attendance[date]:
                writer.writerow([date, student[0], student[1], 'Present'])

    # Step 8: Close the CSV file.
    file.close()



In [3]:
detector = mtcnn.MTCNN()


In [4]:
def normalize(img):
    mean, std = img.mean(), img.std()
    return (img - mean) / std

In [5]:
l2_normalizer = Normalizer('l2')


In [6]:
from tensorflow import keras
face_recogn_model = keras.models.load_model('Rec.h5')
expression_model=keras.models.load_model('FER.h5')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: architecture is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [7]:
class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']
time_period = 2 #Change this value to increase decrease time period
count=0


In [8]:
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)


In [9]:

def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

In [10]:
def load_pickle(path):
    with open(path, 'rb') as f:
        encoding_dict = pickle.load(f)
    return encoding_dict

In [11]:
# Set the time interval for capturing burst images (in seconds)
time_interval = 20

# Set the number of burst images to capture
num_burst_images = 5

# Variables to keep track of time and burst images
start_time = time.time()
current_time = time.time()
burst_images_captured = 0

In [12]:

recognition_t=0.4
confidence_t=0.99

required_size = (160,160)

if __name__ == "__main__":
    required_shape = (160,160)
    face_encoder = face_recogn_model
    expression=expression_model
    path_m = "facenet_keras_weights.h5"
    face_encoder.load_weights(path_m)
    encodings_path = 'encodings/encodings.pkl'
    encoding_dict = load_pickle(encodings_path)
    
    last_prediction_time = time.time()
    time_period = 2 #Change this value to increase decrease time period
    class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']
    people_present=[]
    #capture_interval = 30 # 5 minutes
    cap = cv2.VideoCapture(0)


    while True:

        #frame = cv2.imread('/Users/maazbinkhalid/Desktop/DSC_3553.jpeg')
        labels=[]
        ret, frame = cap.read()  # Capture a frame from the webcam

        # Calculate the elapsed time
        #current_time = time.time()
        #elapsed_time = current_time - start_time

    # Capture burst images every time interval
        
    
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results=detector.detect_faces(img_rgb)
        name=''
        face_num=len(results)
        for res in results:
            if res['confidence'] < confidence_t:
                continue
            x,y,w,h=res['box']
            label_position=(x,y)
            face, pt_1, pt_2 = get_face(img_rgb, res['box'])
            encode = get_encode(face_encoder, face, required_size)
            encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
            name = 'UR'
            distance = float("inf")
            for db_name, db_encode in encoding_dict.items():
                dist = cosine(db_encode, encode)
                if dist < recognition_t and dist < distance:
                    name = db_name
                    distance = dist
                    people_present.append(name)
            print(people_present)
            color = (0, 255, 0) # green color
            thickness = 2
            scale_factor = 1
            new_w = int(w * scale_factor)
            new_h = int(h * scale_factor)
            x_center = int(x + w / 2)
            y_center = int(y + h / 2)
            new_x = int(x_center - new_w / 2)
            new_y = int(y_center - new_h / 2)
            # Draw bounding box on image
            cv2.rectangle(frame, (new_x, new_y), (new_x + new_w, new_y + new_h), color, thickness)
            roi_gray=gray[y:y+h,x:x+w]
            roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
            #Get image ready for prediction
            roi=roi_gray.astype('float')/255.0  #Scale
            roi=img_to_array(roi)
            roi=np.expand_dims(roi,axis=0)  #Expand dims to get it ready for prediction (1, 48, 48, 1)
            preds=expression.predict(roi)[0] 
            #if max(preds)>=0.5:
            label=class_labels[preds.argmax()]  #Find the label
            label_position=(x,y)
            thickness = 2
            font_scale = 0.5 # adjust this parameter to change the font size
# Ge       t the width and height of the text for string1 and string2
            text1_size, _ = cv2.getTextSize(name, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
            text2_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
# Ca       lculate the y-coordinate for the bottom of the first line and the top of the second line
            line1_y = y - 10
            line2_y = line1_y - text1_size[1] - text2_size[1]
# Dr       aw bounding box on image
            #cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
# Ad       d label to image
            cv2.putText(frame, label, (x, line1_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)
            cv2.putText(frame, name, (x, line2_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)
            #cv2.putText(frame,name+"\u2028"+label,label_position,cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            label_position=(x+50,y+50)
        record_attendance(['Maaz','Mustafa','Murtaza','Shaheer','Nofil'],people_present,count)
        print(last_prediction_time)
        label_text = "There are {} students in this frame".format(face_num)
        height, width, channels = frame.shape
        # Define the font, scale, color, and thickness of the text
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 2
        color = (0, 255, 255)  # white color
        thickness = 2
        # Determine the size of the text string
        size = cv2.getTextSize(label_text, font, scale, thickness)
        # Calculate the coordinates of the bottom-left corner of the text string
        x = int((width - size[0][0]) / 2)  # centered horizontally
        y = height - size[0][1] - 10  # 10 pixels above the bottom edge of the image
        # Draw the text string on the image
        start_time = time.time()
        cv2.putText(frame, label_text, (x, y), font, scale, color, thickness, cv2.LINE_AA)
        print('hi')
        cv2.waitKey(1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        cv2.imshow('frame',frame)
    cv2.destroyAllWindows()
            

1/1 [==============================] - 0s 125ms/step


2023-06-06 19:17:40.262429: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 12ms/step
Frame: []
Names: ['Maaz', 'Mustafa', 'Murtaza', 'Shaheer', 'Nofil']
1686061058.770782
hi
1/1 [==============================] - 1s 894ms/step
['Maaz']
1/1 [==============================] - 0s 88ms/step
Frame: ['Maaz']
Names: ['Maaz', 'Mustafa', 'Murtaza', 'Shaheer', 'Nofil']
1686061058.770782
hi
1/1 [==============================] - 0s 53ms/step
['Maaz', 'Maaz']
1/1 [==============================] - 0s 15ms/step
Frame: ['Maaz', 'Maaz']
Names: ['Maaz', 'Mustafa', 'Murtaza', 'Shaheer', 'Nofil']
1686061058.770782
hi
1/1 [==============================] - 0s 36ms/step
['Maaz', 'Maaz', 'Maaz']
1/1 [==============================] - 0s 15ms/step
Frame: ['Maaz', 'Maaz', 'Maaz']
Names: ['Maaz', 'Mustafa', 'Murtaza', 'Shaheer', 'Nofil']
1686061058.770782
hi
1/1 [==============================] - 0s 34ms/step
['Maaz', 'Maaz', 'Maaz', 'Maaz']
1/1 [==============================] - 0s 14ms/step
Frame: ['Maaz', 'Maaz', 'Maaz', 'Maaz']
Names: ['

In [13]:
cap.release()